# Calls RAWG API to get game info

In [38]:
import requests 
import json 
from bs4 import BeautifulSoup

class RAWG:
    def __init__(self):
        self.base_url = 'https://api.rawg.io/api'
        self.session = requests.Session()

    def get_game(self, name):
        game = {}

        id = self.__request_id(name)     
        res = self.__request_game(id)
        
        game['Name'] = res['name']
        game['RawgID'] = res['id']
        game['Metacritic'] = res['metacritic']
        game['Presence'] = self.__parse_precense(res)
        game['Presence'] = self.__score_precense(game['Presence'])
        game['Platform'] = self.__parse_platforms(res['platforms'])
        game['RatingsBreakdown'] = self.__parse_ratings(res['ratings'])
        game['ReleaseDate'] = res['released']
        game['ESRB'] = self.__parse_esrb(res['esrb_rating'])
        game['Achievements'] = res['achievements_count']
        game['CreatorsCount'] = res['creators_count'] 
        game['Description'] = self.__parse_description(res['description'])
        
        return game

    def __request_id(self, name):
        params = { 'search': name }
        res = self.session.get(f'{self.base_url}/games', params=params)
        res = json.loads(res.content)
        return res['results'][0]['id']

    def __request_game(self, id):
        res = self.session.get(f'{self.base_url}/games/{id}')
        res = json.loads(res.content)
        return res
    
    def __parse_description(self, description):
        soup = BeautifulSoup(description)
        return soup.get_text()

    def __parse_precense(self, social_media):
        keys = [ 'reddit_count', 'twitch_count', 'youtube_count', 
                'reviews_text_count', 'ratings_count', 'suggestions_count']
        vals = []

        for key in keys:
            vals.append(social_media[key])
        return dict(zip(keys, vals))
            
    def __score_precense(self, precense):
        score = 0
        for site in precense:
            score += precense[site]
        return score

    def __parse_platforms(self, platforms):
        platforms = [ platform['platform']['name'] for platform in platforms ]
        return ', '.join(platforms )

    def __parse_ratings(self, ratings):
       shortened_ratings =  [ f"{rating['title']}: {rating['count']}" for rating in ratings ]
       return ', '.join(shortened_ratings)
    
    def __parse_esrb(self, esrb_rating):
        if esrb_rating:
            return esrb_rating['name']

## Test

In [42]:
# calls api and returns game info
rawg = RAWG()

rawg.get_game('call of duty')

{'Name': 'Call of Duty',
 'RawgID': 19369,
 'Metacritic': 91,
 'Presence': 1001090,
 'Platform': 'PlayStation 3, Xbox 360, PC, macOS',
 'RatingsBreakdown': 'recommended: 244, exceptional: 116, meh: 29, skip: 8',
 'ReleaseDate': '2003-10-29',
 'ESRB': 'Teen',
 'Achievements': 50,
 'CreatorsCount': 11,
 'Description': "Call of Duty is a World War II first-person shooter developed by Infinity Ward. It is the first game in the series by the same name. \nPlot\nDuring the game's singleplayer campaign, the players take control of three soldiers from different armies: Joey Martin from the U.S. military, sergeant Evans from the British SAS, and Alexei Ivanovich Voronin from the Red Army. The game's plot follows their path from boot camp to the Battle of Berlin. There are 26 missions present in the game.\nGameplay\nCall of Duty features both singleplayer and multiplayer modes. During the singleplayer campaign, a briefing is held or a slide-show with commentary before every mission. In order to c